# CV Simulation — Surface Oxides Au/Ni/Cu

This notebook walks step by step through the cyclic voltammetry simulation code
for metal oxide formation/reduction on an Au/Ni/Cu electrode.

**Model**: surface reaction (Langmuir), no diffusion in solution.  
**Solver**: analytical implicit scheme (numpy), no FEM.  
**Reaction**: $M + H_2O \rightleftharpoons MOH + H^+ + e^-$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Physical Constants

All quantities are in **SI units** (V, A, m², mol/m², s).  
Potentials are expressed **vs Ag/AgCl (sat. KCl)**.

In [ ]:
F = 96485.0        # C/mol  — Faraday constant
R = 8.314          # J/(mol·K)
T = 298.15         # K (25 °C)
n_elec = 1         # electrons per reaction
f = n_elec * F / (R * T)   # nF/RT ≈ 38.9 V⁻¹

A = 1.77e-6        # m² — electrode area (~1.5 mm diameter)
C_dl = 0.30        # F/m² — double layer capacitance (30 µF/cm²)

print(f"f = nF/RT = {f:.2f} V⁻¹")
print(f"RT/F = {R*T/F*1000:.1f} mV  (thermal potential)")

## 2. Metal Parameters (multi-site model)

Each metal has:
- **E0_ox**: oxidation potential(s) — a *list* of sites to create the broad oxidation plateau
- **E0_red**: reduction potential — *single* value (sharp cathodic peak)
- **k0**: surface rate constant [s⁻¹] (not m/s!)
- **Γ_max**: maximum site density [mol/m²]
- **α**: transfer coefficient (symmetric = 0.5)

The **multi-site** model distributes E0_ox across N uniform sites to reproduce
the broad oxidation plateau observed experimentally on gold,
while keeping a single sharp reduction peak.

Here we use **pH 0.3** (H₂SO₄ 0.5 M) as an example.

In [ ]:
# --- Au (gold): reversible oxide, 20 sites between 1.10 and 1.50 V ---
n_sites_Au = 20
E0_ox_Au = np.linspace(1.10, 1.50, n_sites_Au)   # oxidation plateau
frac_sites_Au = np.ones(n_sites_Au) / n_sites_Au  # uniform weights
E0_red_Au = 0.90       # V — single reduction peak
k0_Au = 2.0            # s⁻¹
Gamma_Au = 4.0e-5      # mol/m²
alpha_Au = 0.5
frac_Au = 1.0          # fraction in alloy

print(f"Au: {n_sites_Au} sites, E0_ox = [{E0_ox_Au[0]:.2f}, {E0_ox_Au[-1]:.2f}] V")
print(f"    E0_red = {E0_red_Au:.2f} V")
print(f"    Hysteresis ≈ {(np.mean(E0_ox_Au) - E0_red_Au)*1000:.0f} mV")

## 3. Triangular Potential E(t)

The potentiostat imposes a triangular sweep:

$$E(t) = E_{start} + \nu \cdot t \cdot \text{direction}$$

with $\nu$ = scan rate (0.1 V/s).  
Direction reverses at bounds E_min/E_max or when
current exceeds a threshold (HER/OER walls).

In [ ]:
scan_rate = 0.1    # V/s
E_start = 0.0      # V
E_min = -0.35      # V (cathodic bound)
E_max = 1.65       # V (anodic bound)
dt = 1e-3          # s — time step

def E_waveform(t, E_start, E_min, E_max, scan_rate):
    """Triangular potential: E_start → E_max → E_min → E_start."""
    t1 = (E_max - E_start) / scan_rate   # ramp up
    t2 = (E_max - E_min) / scan_rate     # ramp down
    t3 = (E_start - E_min) / scan_rate   # ramp back
    t_cycle = t1 + t2 + t3
    t_mod = t % t_cycle
    if t_mod < t1:
        return E_start + scan_rate * t_mod
    elif t_mod < t1 + t2:
        return E_max - scan_rate * (t_mod - t1)
    else:
        return E_min + scan_rate * (t_mod - t1 - t2)

# Visualize one cycle
t_demo = np.arange(0, 40, dt)
E_demo = np.array([E_waveform(ti, E_start, E_min, E_max, scan_rate) for ti in t_demo])

plt.plot(t_demo, E_demo)
plt.xlabel('t (s)'); plt.ylabel('E (V vs Ag/AgCl)')
plt.title('Triangular potential — 1 cycle')
plt.grid(True, alpha=0.3)
plt.show()

## 4. Butler-Volmer Kinetics (multi-site)

For each oxidation site $s$ at potential $E^0_{ox,s}$:

$$k_{ox,s} = k_0 \exp\left(\alpha f (E - E^0_{ox,s})\right)$$

Reduction uses a single potential $E^0_{red}$:

$$k_{red} = k_0 \exp\left(-(1-\alpha) f (E - E^0_{red})\right)$$

Exponentials are clamped to ±20 to prevent numerical overflow.

In [ ]:
def butler_volmer_rates(E, E0_ox, E0_red, k0, alpha):
    """Rate constants k_ox and k_red for a given site."""
    eta_ox = E - E0_ox
    eta_red = E - E0_red
    k_ox = k0 * np.exp(np.clip(alpha * f * eta_ox, -20, 20))
    k_red = k0 * np.exp(np.clip(-(1 - alpha) * f * eta_red, -20, 20))
    return k_ox, k_red

# Visualize k_ox, k_red vs E for Au (mean site)
E_range = np.linspace(-0.5, 2.0, 500)
E0_ox_mean = np.mean(E0_ox_Au)
k_ox_demo = np.array([butler_volmer_rates(E, E0_ox_mean, E0_red_Au, k0_Au, alpha_Au)[0] for E in E_range])
k_red_demo = np.array([butler_volmer_rates(E, E0_ox_mean, E0_red_Au, k0_Au, alpha_Au)[1] for E in E_range])

plt.semilogy(E_range, k_ox_demo, 'r-', label=r'$k_{ox}$')
plt.semilogy(E_range, k_red_demo, 'b-', label=r'$k_{red}$')
plt.axvline(E0_ox_mean, color='r', ls=':', alpha=0.5, label=f'E0_ox = {E0_ox_mean:.2f} V')
plt.axvline(E0_red_Au, color='b', ls=':', alpha=0.5, label=f'E0_red = {E0_red_Au:.2f} V')
plt.xlabel('E (V)'); plt.ylabel('k (s⁻¹)')
plt.title('Butler-Volmer rate constants — Au pH 0.3')
plt.legend(); plt.grid(True, alpha=0.3); plt.ylim(1e-6, 1e10)
plt.show()

## 5. Analytical Implicit Scheme for dθ/dt

The Langmuir ODE for surface coverage θ at each site:

$$\frac{d\theta}{dt} = k_{ox}(1-\theta) - k_{red} \cdot \theta$$

This ODE is **linear in θ**, which allows an *exact* implicit scheme:

$$\theta^{n+1} = \frac{\theta^n + \Delta t \cdot k_{ox}}{1 + \Delta t (k_{ox} + k_{red})}$$

This scheme is **unconditionally stable** — no Newton solver needed,
no CFL constraint. We clamp θ ∈ [0, 1] for safety.

In [ ]:
def implicit_step(theta_old, k_ox, k_red, dt):
    """One implicit time step (analytical)."""
    theta_new = (theta_old + dt * k_ox) / (1.0 + dt * (k_ox + k_red))
    return np.clip(theta_new, 0.0, 1.0)

## 6. Current Calculation

The faradaic current for each site $s$ of metal $i$:

$$I_{i,s} = x_i \cdot w_s \cdot n F A \, \Gamma_{max,i} \cdot \frac{d\theta_s}{dt}$$

where $x_i$ = metal fraction in alloy, $w_s$ = site weight.

We add:
- **Capacitive current**: $I_{cdl} = C_{dl} \cdot A \cdot \nu \cdot \text{direction}$
- **HER wall** (cathodic): $I_{HER} = -A \cdot i_{0,HER} \exp(-\alpha_{HER} f \eta_{HER})$
- **OER wall** (anodic): $I_{OER} = A \cdot i_{0,OER} \exp((1-\alpha_{OER}) f \eta_{OER})$

In [ ]:
# HER/OER parameters at pH 0.3
pH = 0.3
E0_HER = -0.10 - 0.059 * pH    # V — cathodic wall
E0_OER = 1.50 - 0.059 * pH     # V — anodic wall
i0_HER = 0.05     # A/m²
i0_OER = 0.15     # A/m²
alpha_HER = 0.5
alpha_OER = 0.5
I_threshold = 7e-6  # A — scan reversal threshold

print(f"pH {pH}: E_HER = {E0_HER:.3f} V, E_OER = {E0_OER:.3f} V")
print(f"Water window = {E0_OER - E0_HER:.3f} V")

## 7. Full Simulation — Pure Au, pH 0.3

The time loop simultaneously integrates:
1. The potential E(t) with dynamic steering (reversal at current threshold)
2. Surface coverage θ for each site (implicit scheme)
3. Total current (faradaic + capacitive + HER/OER)

In [ ]:
# Allocation
n_cycles = 2
t_cycle = 2 * (E_max - E_min) / scan_rate
n_steps = int(n_cycles * t_cycle / dt)

t_arr = np.zeros(n_steps)
E_arr = np.zeros(n_steps)
I_arr = np.zeros(n_steps)
theta_arr = np.zeros((n_steps, n_sites_Au))  # θ per site

# Initial state
E_current = E_start
scan_dir = +1         # +1 = anodic, -1 = cathodic
phase = 0             # 0: ramp up, 1: ramp down, 2: return
cycles_done = 0
n_stab = 100          # stabilization steps (ignore transient)

for step in range(n_steps - 1):
    t_arr[step] = step * dt
    E_arr[step] = E_current
    E_next = E_current + scan_rate * dt * scan_dir

    I_step = 0.0

    # --- 1. Oxide current (each site) ---
    for s in range(n_sites_Au):
        k_ox, k_red = butler_volmer_rates(
            E_next, E0_ox_Au[s], E0_red_Au, k0_Au, alpha_Au)
        theta_old = theta_arr[step, s]
        theta_new = implicit_step(theta_old, k_ox, k_red, dt)
        theta_arr[step + 1, s] = theta_new
        dtheta_dt = (theta_new - theta_old) / dt
        I_step += frac_Au * frac_sites_Au[s] * n_elec * F * A * Gamma_Au * dtheta_dt

    # --- 2. HER wall ---
    eta_HER = E_current - E0_HER
    I_step += -A * i0_HER * np.exp(np.clip(-alpha_HER * f * eta_HER, -30, 30))

    # --- 3. OER wall ---
    eta_OER = E_current - E0_OER
    I_step += A * i0_OER * np.exp(np.clip((1 - alpha_OER) * f * eta_OER, -30, 30))

    # --- 4. Capacitive current ---
    I_step += C_dl * A * scan_rate * scan_dir

    I_arr[step] = I_step

    # --- 5. Potential steering ---
    if step >= n_stab:
        if scan_dir == +1 and I_step > +I_threshold:
            scan_dir = -1; phase = 1
        elif scan_dir == -1 and I_step < -I_threshold:
            scan_dir = +1; phase = 2

    # Safety bounds
    if scan_dir == +1 and E_current >= E_max:
        scan_dir = -1; phase = 1
    elif scan_dir == -1 and E_current <= E_min:
        scan_dir = +1; phase = 2

    # Cycle end detection
    if phase == 2 and scan_dir == +1 and E_current >= E_start:
        cycles_done += 1
        if cycles_done >= n_cycles:
            t_arr = t_arr[:step+2]; E_arr = E_arr[:step+2]
            I_arr = I_arr[:step+2]; theta_arr = theta_arr[:step+2]
            break
        phase = 0

    E_current += scan_rate * dt * scan_dir

print(f"Simulation complete: {len(t_arr)} points, {cycles_done} cycles")
print(f"I_max = {np.max(I_arr)*1e6:.1f} µA,  I_min = {np.min(I_arr)*1e6:.1f} µA")

## 8. Cyclic Voltammogram I(E)

Plot current density $j$ (mA/cm²), masking regions where $|I|$
exceeds the threshold (HER/OER walls) and the initial transient.

In [ ]:
A_cm2 = A * 1e4  # m² → cm²
j = I_arr * 1e3 / A_cm2             # mA/cm²
j_threshold = I_threshold * 1e3 / A_cm2

mask = (np.abs(j) <= j_threshold)
mask[:n_stab] = False  # mask transient

fig, ax = plt.subplots(figsize=(10, 7))
for i in range(len(t_arr) - 1):
    if mask[i] and mask[i+1]:
        color = 'tab:red' if E_arr[i+1] > E_arr[i] else 'tab:blue'
        ax.plot(E_arr[i:i+2], j[i:i+2], color=color, lw=1.5)

ax.axhline(0, color='gray', ls='--', alpha=0.5)
ax.axhline(j_threshold, color='red', ls=':', alpha=0.4, label=f'Threshold ±{j_threshold:.1f} mA/cm²')
ax.axhline(-j_threshold, color='red', ls=':', alpha=0.4)
ax.set_xlabel('E (V vs Ag/AgCl)'); ax.set_ylabel('j (mA/cm²)')
ax.set_title(f'CV — Pure Au, pH {pH}, ν = {scan_rate} V/s')
ax.set_xlim(E_arr[mask].min() - 0.05, E_arr[mask].max() + 0.05)
ax.set_ylim(-1.2 * j_threshold, 1.2 * j_threshold)
ax.legend(loc='upper left'); ax.grid(True, alpha=0.3)
plt.tight_layout(); plt.show()

## 9. Surface Coverage θ(E)

The mean coverage $\bar{\theta} = \sum_s w_s \theta_s$ shows the
characteristic hysteresis: oxidation occurs over a broad plateau
(multi-site) while reduction is a sharp peak (single E0_red).

In [ ]:
theta_mean = theta_arr @ frac_sites_Au  # weighted average across sites

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(E_arr, theta_mean, 'gold', lw=2)
ax1.set_xlabel('E (V)'); ax1.set_ylabel('θ')
ax1.set_title('Coverage θ(E)'); ax1.set_ylim(-0.05, 1.05)
ax1.grid(True, alpha=0.3)

ax2.plot(t_arr, theta_mean, 'gold', lw=2)
ax2.set_xlabel('t (s)'); ax2.set_ylabel('θ')
ax2.set_title('Coverage θ(t)'); ax2.set_ylim(-0.05, 1.05)
ax2.grid(True, alpha=0.3)

plt.tight_layout(); plt.show()

## 10. What the Voltammogram Shows

| Feature | Physical origin |
|---------|----------------|
| **Broad anodic plateau** | Multi-site: 20 E0_ox distributed between 1.10 and 1.50 V |
| **Sharp cathodic peak** | Single E0_red at 0.90 V — all oxides reduce at the same potential |
| **Hysteresis** | E0_ox ≠ E0_red — thermodynamic irreversibility of the oxide |
| **Vertical offset** | Capacitive current I_cdl = C_dl × A × ν |
| **Edge cutoff** | Current threshold = HER/OER walls |

---

*Source code: `cv_surface_oxide.py` + `parameters_oxide.py`*  
*Dependencies: numpy, matplotlib — no FEM.*